<h1>Job Searcher Indeed </h1>


Before we used "requests" library but as indeed is now part of cloudflare we need to use a library as cloudscraper to bypass that wee security.

In [1]:
#import requests
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd

We eliminate the headers as they are not necesary for this proyect, as before they was for the requests library.

* headers={"user_Agents":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}

In [2]:
def extract_the(page=0,word1="assistant",word2="manager"):
    scraper = cloudscraper.create_scraper(delay=10, browser='chrome')
    url=f'https://uk.indeed.com/jobs?q={word1}+{word2}&start={page}' # placeholder to go throw the information we need
    r = scraper.get(url)
    soup= BeautifulSoup(r.text,"html.parser")
    return soup,r
print(extract_the()[1])

<Response [200]>


**We also add the response in the last cell, to catch is is a problem with the update of the cloudflare as the last time and we are not conecting properly.**

In [3]:
def transform(soup):

    jobs=soup.find_all("div",class_="job_seen_beacon") # the list that it gives us back is each of the jobs
    for job in jobs: # we go throw the list finding what we need and adding all of this to a dictionary
        djobs={
        "Title"      :job.find("a").text.strip(),
        "Company"    :job.find("span",class_="companyName").text.strip(),
        "Location"   :job.find("div",class_="companyLocation").text.strip(),
        "Link"       :"https://uk.indeed.com/viewjob?jk="+ job.find("a").attrs["data-jk"],
        "Status"     :job.find("span",class_="date").text.strip()
        }
        try: # some of them do not have salary so to try evade errors we do a try and except
            djobs["salary"]=job.find("div",class_="metadata salary-snippet-container").text.strip()
        except:
            djobs["salary"]="Not Declared"
        try: # this sometimes give error, i can not find the problem yet, so we leave here
            djobs["Description"]=job.find("li").text.strip()
        except:
            djobs["Description"]="Not Found"
            
        joblist.append(djobs)
    return 

joblist=[] 
            
print("done")

done


In [4]:
for pages in range(0,10): # run throw the first 10 pages
    info=extract_the(pages)[0]
    transform(info)

#print(joblist)

In [5]:
df=pd.DataFrame(joblist) #creating a dataframe

In [6]:
df.head(10)# head of the data frame

,Title,Company,Location,Link,Status,salary,Description
0,Assistant Store Manager,DFS Furniture Stores,Glasgow,https://uk.indeed.com/viewjob?jk=a477287423f067d9,PostedPosted 2 days ago,"£65,000 a year","They coach, motivate, and inspire their collea..."
1,Assistant Venue Manager,Vega,Glasgow G2,https://uk.indeed.com/viewjob?jk=e7bfca102a9f49a8,PostedPosted 15 days ago,"£26,000 a year",We are looking for someone who loves to coach ...
2,Assistant manager,Gamba,Glasgow,https://uk.indeed.com/viewjob?jk=c9c5dda56c4d4d58,PostedPosted 2 days ago,"£27,000 a year",You would have responsability for assistaning ...
3,Assistant Manager,Savoy Nightclub,Glasgow G2,https://uk.indeed.com/viewjob?jk=cff3fe3553c380e5,EmployerActive 3 days ago,"£21,000 - £23,000 a year",Day to day running of the business.
4,Assistant Manager,HY Hospitality Group,Glasgow G2,https://uk.indeed.com/viewjob?jk=130b741f7def4cab,PostedToday,"From £24,000 a year",45 hour contracted working week.
5,Assistant General Manager - Glasgow,The Alchemist Bar & Restaurant,Glasgow+2 locations,https://uk.indeed.com/viewjob?jk=550792536ff3fcb0,EmployerActive 6 days ago,"From £30,000 a year","You will begin to learn, Profits & Loss, Sales..."
6,General and Assistant General Manager,Wonder World Group,Glasgow,https://uk.indeed.com/viewjob?jk=4fd78a1f5d8a4bb0,PostedToday,"£22,000 - £30,000 a year","Required to work non traditional hours, weeken..."
7,Assistant Manager,Blue Lagoon Fish & Chips,Glasgow,https://uk.indeed.com/viewjob?jk=10d10a7b11ba4d89,PostedPosted 2 days ago,From £11 an hour,You will be fully trained in all aspects which...
8,Assistant Manager - Flannels,Flannels,Glasgow G34,https://uk.indeed.com/viewjob?jk=6cb507636987a6da,PostedPosted 30+ days ago,"£25,250 - £35,000 a year","Trend aware, fashion conscious and well presen..."
9,Assistant Manager,ABERDAM Dutch Fries,Glasgow G5,https://uk.indeed.com/viewjob?jk=443c86b4cd2f54d1,PostedToday,"£21,665 - £23,000 a year",Our flagship site see over 1000 orders a week ...


In [7]:
df.to_excel('GlasgowJobs.xlsx', sheet_name='AssistantManager')